In [36]:
import pandas as pd
import numpy as np
import yfinance as yf
import json
import threading
import logging

logging.basicConfig(level=logging.INFO, filename='load_data.log')
tickersInfo=[] # all tickers Information downloaded from yahoo
batchSize=300  # number of tickers being processed in a thread
  
def getValue(info, key):
    if(key in info): 
        return info[key]
    else:
        return np.nan
    
def save2csv(filename):
    selected_keys= yf.Ticker('msft').info.keys()
    tickersFrame = pd.DataFrame(data=[],columns=selected_keys)

    for t in tickersInfo:
        s={key: getValue(t, key) for key in selected_keys}
        tickersFrame=tickersFrame.append(s, ignore_index=True)

    tickersFrame.set_index(['symbol'], inplace=True)
    tickersFrame.sort_index(inplace=True)
    tickersFrame.to_csv(filename)
    
def appendToTickersInfo(arange):
    global tickersInfo
    for t in arange:
        try:
            tickersInfo.append(hk_tickers.tickers[t].info)
            if (t%100==0): logging.info("Adding tickers[" + str(t) + "]")
        except:
            pass

class DownloadThread (threading.Thread):
    def __init__(self, threadID, name, arange):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.arange = arange
    def run(self):
      logging.info("Starting " + self.name)
      appendToTickersInfo(self.arange)
      logging.info("Exiting " + self.name)

def loadTickersInfoFromYahoo(tickerscsv, outcsv):
    hk_tickers=pd.read_csv(tickerscsv)
    hk_tickers=yf.Tickers(hk_tickers.Tickers.to_list())
    logging.info('There are ' + str(len(hk_tickers.tickers)) + ' tickers to be loaded in background threads.')
    threads = [DownloadThread(i, 'T'+str(i), range(batchSize*i,batchSize*(i+1))) for i in range(int(len(hk_tickers.tickers) / batchSize)+1)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    save2csv(outcsv)

In [37]:
loadTickersInfoFromYahoo('HKTickers.csv', 'HKTickersInfo.csv')